In [33]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import itertools
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 800)

base_url = 'https://www.evarzytynes.lt'

escapes = ''.join([chr(char) for char in range(1, 32)])

def auction_parser(url):
    auction_page = urlopen(url)
    auction_soup = BeautifulSoup(auction_page, 'html.parser')
    max_price_so_far = auction_soup.find('span', attrs={'id': 'maxBid'}).contents[0].strip()
    
    auction_desc = auction_soup.find('div', attrs={'class': 'load_block'})
    product_descs = auction_soup.find_all('div', attrs={'class': 'product_desc'})
    for product_desc in product_descs:

        product_property_tuples = list(zip(product_desc.find('ul', attrs={'class': 'list'}).find_all('span', attrs={'class': 'left'}),
                product_desc.find('ul', attrs={'class': 'list'}).find_all('span', attrs={'class': 'right'})))

        auction_property_tuples = list(zip(auction_desc.find('ul', attrs={'class': 'list'}).find_all('span', attrs={'class': 'left'}),
                auction_desc.find('ul', attrs={'class': 'list'}).find_all('span', attrs={'class': 'right'})))

        properties = {}
        for product_property_tuple in product_property_tuples:
            properties.setdefault(product_property_tuple[0].text.strip(), product_property_tuple[1].text.strip())

        for auction_property_tuple in auction_property_tuples:
            properties.setdefault(auction_property_tuple[0].text.strip(), auction_property_tuple[1].text.strip())

        properties.setdefault('url', url)
        yield properties

def auction_list_parser():
    content_exists = True
    result = []
    page_id = 0

    while content_exists:
        url = f'{base_url}/evs/pages/auctions.do?estateType=1&estateSubtype=1&stateType=PASKELBTA-IR-VYKSTA&page={page_id}'
        page = urlopen(url)
        soup = BeautifulSoup(page, 'html.parser')

        sub_result = soup.find_all('div', attrs={'class': 'no'})
        if sub_result:
            result.append(sub_result)
            page_id += 1
            print(f'Reading page {page_id}')

        else:
            content_exists = False
    return result
    
if __name__ == '__main__':
    auction_urls = list(itertools.chain(*auction_list_parser()))
    print(f'Found {len(auction_urls)} auctions')


    result = []
    for i in auction_urls:
        auction_url = base_url + i.find('a')['href']
        for auction_object in auction_parser(auction_url):
            result.append(pd.DataFrame.from_dict(auction_object, orient='index').T)

    if result:    
        final = pd.concat(result)

        final['Bendras plotas:'] = final['Bendras plotas:'].apply(lambda x: str(x).translate(escapes).replace('\n', '').replace('\r', ''))
        final['Užstatytas plotas:'] = final['Užstatytas plotas:'].apply(lambda x: str(x).translate(escapes).replace('\n', '').replace('\r', ''))

        final.to_excel('evarzytines.xlsx', index=False)


Reading page 1
Reading page 2
Reading page 3
Reading page 4
Reading page 5
Reading page 6
Reading page 7
Reading page 8
Reading page 9
Reading page 10
Reading page 11
Reading page 12
Reading page 13
Reading page 14
Reading page 15
Reading page 16
Reading page 17
Reading page 18
Reading page 19
Reading page 20
Reading page 21
Reading page 22
Reading page 23
Found 455 auctions


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:70: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [34]:
auction_size = (final
                .groupby('url')
                ['Turto potipis:']
                .count()
                .reset_index()
                .rename(columns={'Turto potipis:': 'Auction size'}))

In [52]:
def parsed_area(x):
    result = (''.join(c for c in x.split('a.')[0] if c.isdigit() or c in [','])).replace(',', '.')
    if result:
        return float(result)
    return None


final['Pradinė pardavimo kaina, Eur:'] = (final['Pradinė pardavimo kaina:']
                                     .apply(lambda x: float(''.join(c for c in x if c.isdigit())
                                                           )
                                           )
                                    )

final['Bendras plotas (a):'] = (final['Bendras plotas:']
                                     .apply(lambda x: parsed_area(x)
                                           )
                                    )

final['Vieno aro pradine kaina, Eur:'] = final['Pradinė pardavimo kaina, Eur:'] / final['Bendras plotas (a):']
final = final.merge(auction_size, on='url', how='left')

In [53]:
# filtering
(final[(final['Turto potipis:'] == 'Sklypai') &
     (final['Parduodama turto dalis:'] == '1/1') &
      (final['Vieno aro pradine kaina, Eur:'] < 15) & 
      (final['Bendras plotas (a):'] > 150) & 
      (final['Auction size'] == 1)]
 .sort_values('Vieno aro pradine kaina, Eur:')
[['Vieno aro pradine kaina, Eur:',
  'Bendras plotas:',
  'Adresas:', 
 'Pradinė pardavimo kaina:',
  'Pabaigos data:',
  'Būsena:',
       'Dalyvio mokestis:',
  'url',
  'Dalyvio registracijos tvirtinimo trukmė d. d.:',
       'Garantinis įnašas:',
  'Interneto nuoroda:', 
  'Kadastrinis Nr.:',
       'Kambarių/patalpų skaičius:', 
   'Aprašymas:',
  'Pagrindas',
       'Parduodama turto dalis:',
       'Pradinė NT pardavimo kaina iš bendros pradinės pardavimo kainos:',
       
       'Pradinė žemės sklypo pardavimo kaina iš bendros pradinės pardavimo kainos:',
       'Pradinė žemės sklypo vertė žemės sklypo mokesčiui apskaičiuoti:',
       'Pradžios data:', 'Registracijos pabaiga:', 'Registracijos pradžia:',
       'Registravimo mokestis:', 'Savininkai:', 'Sklypas:', 'Tipas:',
       'Turtas PVM objektas:', 'Turto nuosavybės apribojimai:',
       'Turto potipis:', 'Turto sąrašas:', 'Unikalus Nr.:',
       'Užstatytas plotas:', 'Bendras plotas (a):'
       ]]
)

,"Vieno aro pradine kaina, Eur:",Bendras plotas:,Adresas:,Pradinė pardavimo kaina:,Pabaigos data:,Būsena:,Dalyvio mokestis:,url,Dalyvio registracijos tvirtinimo trukmė d. d.:,Garantinis įnašas:,Interneto nuoroda:,Kadastrinis Nr.:,Kambarių/patalpų skaičius:,Aprašymas:,Pagrindas,Parduodama turto dalis:,Pradinė NT pardavimo kaina iš bendros pradinės pardavimo kainos:,Pradinė žemės sklypo pardavimo kaina iš bendros pradinės pardavimo kainos:,Pradinė žemės sklypo vertė žemės sklypo mokesčiui apskaičiuoti:,Pradžios data:,Registracijos pabaiga:,Registracijos pradžia:,Registravimo mokestis:,Savininkai:,Sklypas:,Tipas:,Turtas PVM objektas:,Turto nuosavybės apribojimai:,Turto potipis:,Turto sąrašas:,Unikalus Nr.:,Užstatytas plotas:,Bendras plotas (a):
465,1.636893,"1030  a.  (10,3 ha.)",Rokiškio r. sav. Audronių I k.,1 686 Eur,2019-11-07 09:00:59,Vykstančios\nBankroto administratorių varžytynės,"168,60 Eur",https://www.evarzytynes.lt/evs/pages/auction.do?id=213187&number=179301,NaN,NaN,,7325/0002:360,NaN,"Adresas - Rokiškio r. sav., Obelių sen., Audronių I k., paskirtis - žemės ūkio, plotas - 10,30 ha. \nPASTABA: Asmenims, siekiantiems įsigyti žemės ūkio paskirties žemės, taikomi LR Žemės ūkio paskirties žemės įsigijimo įstatyme nustatyti reikalavimai.",176080,1/1,NaN,NaN,NaN,2019-10-08 09:00:00,NaN,NaN,NaN,"ŽŪB ""Kraštovaizdis"", 300140724",NaN,Antrosios,Ne,Įkeistas,Sklypai,NaN,4400-1008-8774,nan,1030.00
426,3.023061,"477  a.  (4,77 ha.)",Kupiškio r. sav. Punkiškių k.,1 442 Eur,2019-11-04 10:39:59,Vykstančios\nAntstolių varžytynės,"144,20 Eur",https://www.evarzytynes.lt/evs/pages/auction.do?id=213248&number=179363,NaN,NaN,NaN,5701/0003:157,NaN,Žemės sklypo kadastro numeris ir kadastro vietovės pavadinimas: 5701/0003:157 Adomynės k.v. \r\nDaikto pagrindinė naudojimo paskirtis: Žemės ūkio \r\nŽemės sklypo naudojimo būdas: Kiti žemės ūkio paskirties žemės sklypai \r\nŽemės sklypo plotas: 4.7700 ha \r\nVandens telkinių plotas: 0.0400 ha \r\nKitos žemės plotas: 4.7300 ha \r\nŽemės ūkio naudmenų našumo balas: 13.0 \r\nMatavimų tipas: Žemės sklypas suformuotas atliekant preliminarius matavimus,NaN,1/1,NaN,NaN,NaN,2019-10-04 10:39:01,NaN,NaN,NaN,"UAB ""Aves ūkis"", 301112717",NaN,Pirmosios,NaN,Disponavimo teisės apribojimas\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\tHipotekos nutartis,Sklypai,NaN,4400-1549-1788,nan,477.00
423,3.023841,"1510  a.  (15,1 ha.)",Panevėžio r. sav. Šilų k.,4 566 Eur,2019-11-04 10:39:59,Vykstančios\nAntstolių varžytynės,"456,60 Eur",https://www.evarzytynes.lt/evs/pages/auction.do?id=213242&number=179357,NaN,NaN,NaN,6679/0001:307,NaN,Žemės sklypo kadastro numeris ir kadastro vietovės pavadinimas: 6679/0001:307 Šventupių k.v. \r\nDaikto pagrindinė naudojimo paskirtis: Žemės ūkio \r\nŽemės sklypo naudojimo būdas: Kiti žemės ūkio paskirties žemės sklypai \r\nŽemės sklypo plotas: 15.1000 ha \r\nKitos žemės plotas: 15.1000 ha \r\nŽemės ūkio naudmenų našumo balas: 20.8 \r\nMatavimų tipas: Žemės sklypas suformuotas atliekant preliminarius matavimus,NaN,1/1,NaN,NaN,NaN,2019-10-04 10:39:01,NaN,NaN,NaN,"UAB ""Aves ūkis"", 301112717",NaN,Pirmosios,NaN,Disponavimo teisės apribojimas\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\tHipotekos nutartis,Sklypai,NaN,4400-1218-0415,nan,1510.00
424,3.024499,"1347  a.  (13,47 ha.)",Kupiškio r. sav. Punkiškių k.,4 074 Eur,2019-11-04 10:39:59,Vykstančios\nAntstolių varžytynės,"407,40 Eur",https://www.evarzytynes.lt/evs/pages/auction.do?id=213244&number=179359,NaN,NaN,NaN,5701/0002:304,NaN,Žemės sklypo kadastro numeris ir kadastro vietovės pavadinimas: 5701/0002:304 Adomynės k.v. \r\nDaikto pagrindinė naudojimo paskirtis: Žemės ūkio \r\nŽemės sklypo naudojimo būdas: Kiti žemės ūkio paskirties žemės sklypai \r\nŽemės sklypo plotas: 13.4700 ha \r\nVandens telkinių plotas: 0.2900 ha \r\nKitos žemės plotas: 13.1800 ha \r\nŽemės ūkio naudmenų našumo balas: 13.0 \r\nMatavimų tipas: